<a href="https://colab.research.google.com/github/Yajie-Xu/HuggingFace-Course-Agents/blob/main/unit1/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from smolagents import CodeAgent,DuckDuckGoSearchTool, HfApiModel,load_tool,tool
import datetime
import requests
import pytz
import yaml
from tools.final_answer import FinalAnswerTool

from Gradio_UI import GradioUI

# Below is an example of a tool that does nothing. Amaze us with your creativity !
@tool
def food_nutrition(food: str) -> str:
    """A tool that takes a food name as input and performs a web search
    to find its calories, fat, carbs, and protein per 100g.

    Args:
        food: A string representing a food item.

    Returns:
        A formatted string containing nutritional information.
    """
    try:
        search_tool = DuckDuckGoSearchTool()  # Requires smolagents
        query = f"Nutritional facts per 100g of {food} (calories, fat, carbs, protein)"
        results = search_tool(query)

        if results:
            # You will need to extract actual values from the results.
            # For now, here are mock placeholders:
            calories = "XXX kcal"
            fat = "YY g"
            carbs = "ZZ g"
            protein = "WW g"

            return (
                f"Nutritional information for 100g of {food}:\n"
                f" * Calories: {calories}\n"
                f" * Fat: {fat}\n"
                f" * Carbs: {carbs}\n"
                f" * Protein: {protein}"
            )
        else:
            return f"Sorry, I couldn't find any nutritional data for {food}."

    except Exception as e:
        return f"Error fetching nutritional facts for '{food}': {e}. Please check the spelling or try another food."

# def my_custom_tool(arg1:str, arg2:int)-> str: #it's import to specify the return type
#     #Keep this format for the description / args / args description but feel free to modify the tool
#     """A tool that does nothing yet
#     Args:
#         arg1: the first argument
#         arg2: the second argument
#     """
#     return "What magic will you build ?"

# def food_nutrition(food: str) -> str:
#     """A tool that takes a food name as an input and performs a web search to find its calories, fat, carbs and protein.
#     Args:
#         food: a string representing a food
#     """
#     try:
#         search_tool = DuckDuckGoSearchTool() # Doc: Agents - Guided tour: https://huggingface.co/docs/smolagents/en/guided_tour
#         query = f"What are the calories, fat, carbs and protein of one 100g of this {food}"
#         results = search_tool(query)

#         if results:
#             return f"The nutrition factor of {food} per 100g is:\\
#             \n * Calories: results_calory;\\
#             \n * Carbs: results_fat;\\
#             \n * Protein: results_potein.\\
#             \n * Fat: results_fat;"

#         return f"You've stumped me - I don't know what the best pizza in {city} could be."
#     except Exception as e:
#         return f'Error fetching the nutrition fact for {food}. Could you exman your spelling or provide me another food?'

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"


final_answer = FinalAnswerTool()

# If the agent does not answer, the model is overloaded, please use another model or the following Hugging Face Endpoint that also contains qwen2.5 coder:
# model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud'

model = HfApiModel(
max_tokens=2096,
temperature=0.5,
model_id='Qwen/Qwen2.5-Coder-32B-Instruct',# it is possible that this model may be overloaded
custom_role_conversions=None,
)


# Import tool from Hub
image_generation_tool = load_tool("agents-course/text-to-image", trust_remote_code=True)

with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)

agent = CodeAgent(
    model=model,
    tools=[final_answer], ## add your tools here (don't remove final answer)
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)


GradioUI(agent).launch()

ModuleNotFoundError: No module named 'smolagents'